<a href="https://colab.research.google.com/github/hws2002/Deep_Learning_with_Keras/blob/main/Chapter2/2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# implement model from scratch

단순한 Dense 클래스

In [11]:
import tensorflow as tf

class NaiveDense:
  def __init__(self, input_size, output_size, activation):
    self.activation = activation

    w_shape = (input_size, output_size)
    w_initial_value = tf.random.uniform(w_shape, minval = 0, maxval = 1e-1)
    self.W = tf.Variable(w_initial_value)

    b_shape = (output_size,)
    b_initial_value = tf.zeros(b_shape)
    self.b = tf.Variable(b_initial_value)

  def __call__(self, inputs):
    return self.activation(tf.matmul(inputs, self.W) + self.b)

  @property
  def weights(self):
    return [self.W, self.b]

단순한 Sequential 클래스

In [12]:
class NaiveSequential:
  def __init__(self, layers):
    self.layers = layers

  def __call__(self, inputs):
    x = inputs
    for layer in self. layers:
      x = layer(x)
    return x

  @property
  def weights(self):
    weights = []
    for layer in self.layers:
      weights += layer.weights
    return weights


In [13]:
model = NaiveSequential([
    NaiveDense(input_size = 28*28, output_size = 512, activation = tf.nn.relu),
    NaiveDense(input_size = 512, output_size = 10, activation = tf.nn.softmax)
])

assert len(model.weights) == 4

배치 제너레이터
MNIST 데이터를 미니 배치로 순회하기 위해, 배치 제너레이터가 필요

In [60]:
import math

class BatchGenerator:
  def __init__(self, images, labels, batch_size = 128):
    assert len(images) == len(labels)
    self.index = 0
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.num_batches = math.ceil(len(images) / batch_size)

  def next(self):
    images = self.images[self.index : self.index + self.batch_size]
    labels = self.labels[self.index : self.index + self.batch_size]
    self.index += self.batch_size
    return images, labels

In [58]:
# 이렇게 구현될때도 있지 않나..?
import math

class BatchGenerator:
  def __init__(self, images, labels, batch_size = 128):
    assert len(images) == len(labels)
    self.index = 0
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.num_batches = math.floor(len(images) / batch_size)

  def next(self):
    images = self.images[self.index : self.index + self.batch_size]
    labels = self.labels[self.index : self.index + self.batch_size]
    if self.index // self.batch_size == self.num_batches - 1:
      images = self.images[self.index:]
      labels = self.images[self.index:]
    self.index += self.batch_size
    return images, labels

In [61]:
import numpy as np
x = tf.ones((260, 10))
y = tf.zeros((260,))
print(len(x))
print(len(y))

260
260


In [63]:
bg = BatchGenerator(x,y)
for _ in range(bg.num_batches):
  images, labels = bg.next()
  print(len(images), len(labels))

128 128
128 128
4 4


# 훈련 스텝 실행하기

In [69]:
learning_rate = 1e-3

def update_weights(gradients, weights):
  for g,w in zip(gradients, weights):
    w.assign_sub(g*learning_rate) # 텐서플로 변수의 assign_sub 메서드는 -=와 동일함

def one_training_step(model, images_batch, labels_batch):
  with tf.GradientTape() as tape:
    predictions = model(images_batch)
    per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
        labels_batch, predictions)
    average_loss = tf.reduce_mean(per_sample_losses) # 정방향 패스 실행
  gradients = tape.gradient(average_loss, model.weights) # 가중치에 대한 손실의 그레디언트 계산,
                                                         # gradients 리스트의 각 항목은 model.weights 리스트에 있는 가중치에 매칭됨

  update_weights(gradients, model.weights) # 그레디언트 사용하여 가중치 업데이트
  return average_loss

In [70]:
from tensorflow.keras import optimizers

optimizer = optimizers.SGD(learning_rate=1e-3)

def update_weights(gradients, weights):
  optimizer.apply_gradients(zip(gradients, weights))

# 전체 훈련 루프

In [71]:
def fit(model, images, labels, epochs, batch_size = 128):
  for epoch_counter in range(epochs):
    print(f"에포크 {epoch_counter}")
    batch_generator = BatchGenerator(images, labels)
    for batch_counter in range(batch_generator.num_batches):
      images_batch, labels_batch = batch_generator.next()
      loss = one_training_step(model, images_batch, labels_batch)
      if batch_counter % 100 == 0:
        print(f"{batch_counter}번째 배치 손실 : {loss:.2f}")

In [72]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28*28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype("float32") / 255

fit(model, train_images, train_labels, epochs = 10, batch_size = 128)

에포크 0
0번째 배치 손실 : 5.50
100번째 배치 손실 : 2.26
200번째 배치 손실 : 2.21
300번째 배치 손실 : 2.09
400번째 배치 손실 : 2.20
에포크 1
0번째 배치 손실 : 1.90
100번째 배치 손실 : 1.90
200번째 배치 손실 : 1.83
300번째 배치 손실 : 1.71
400번째 배치 손실 : 1.81
에포크 2
0번째 배치 손실 : 1.58
100번째 배치 손실 : 1.60
200번째 배치 손실 : 1.51
300번째 배치 손실 : 1.43
400번째 배치 손실 : 1.50
에포크 3
0번째 배치 손실 : 1.33
100번째 배치 손실 : 1.36
200번째 배치 손실 : 1.24
300번째 배치 손실 : 1.21
400번째 배치 손실 : 1.27
에포크 4
0번째 배치 손실 : 1.13
100번째 배치 손실 : 1.17
200번째 배치 손실 : 1.05
300번째 배치 손실 : 1.05
400번째 배치 손실 : 1.11
에포크 5
0번째 배치 손실 : 0.99
100번째 배치 손실 : 1.03
200번째 배치 손실 : 0.91
300번째 배치 손실 : 0.93
400번째 배치 손실 : 0.99
에포크 6
0번째 배치 손실 : 0.88
100번째 배치 손실 : 0.93
200번째 배치 손실 : 0.80
300번째 배치 손실 : 0.84
400번째 배치 손실 : 0.90
에포크 7
0번째 배치 손실 : 0.80
100번째 배치 손실 : 0.84
200번째 배치 손실 : 0.72
300번째 배치 손실 : 0.77
400번째 배치 손실 : 0.83
에포크 8
0번째 배치 손실 : 0.74
100번째 배치 손실 : 0.77
200번째 배치 손실 : 0.66
300번째 배치 손실 : 0.71
400번째 배치 손실 : 0.78
에포크 9
0번째 배치 손실 : 0.69
100번째 배치 손실 : 0.72
200번째 배치 손실 : 0.61
300번째 배치 손실 : 0.67
400번째 배치 손실 : 0.74


# 모델 평가하기

In [75]:
predictions = model(test_images).numpy()
predicted_labels = np.argmax(predictions, axis = 1)
matches = predicted_labels == test_labels

print(f"정확도 : {matches.mean():.2f}")

정확도 : 0.81
